In [2]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg
#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#inatll opencv in python
#!pip3 install opencv-python
import cv2

In [6]:
pip install opencv-python

     |████████████████████████████████| 60.5 MB 11.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [34]:
from glob import glob
from tqdm.notebook import tqdm

fd = "/home/humble_local2_25t/alexw/MIST_round3/Human_DLPFC_ARI/"
fns = glob(f"{fd}/tissue_*")

out_fd = "/home/humble_local2_25t/alexw/MIST_round3/MIST_manuscript_revision/Human_DLPFC_ARI/region_detection_results/"
for i, fn in enumerate(tqdm(fns)):
    tissue_id = fn.split("/")[-1]
    fn = f"{fd}{tissue_id}/data.h5ad"
    adata=sc.read(fn)
    #Set coordinates
    x_array=adata.obs["array_col"].tolist()
    y_array=adata.obs["array_row"].tolist()
    x_pixel=adata.obs["pxl_col_in_fullres"].tolist()
    y_pixel=adata.obs["pxl_row_in_fullres"].tolist()
    s=1
    b=49
    adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, histology=False)
    adata.var_names_make_unique()
    spg.prefilter_genes(adata,min_cells=3) # avoiding all genes are zeros
    spg.prefilter_specialgenes(adata)
    #Normalize and take log for UMI
    sc.pp.normalize_per_cell(adata)
    sc.pp.log1p(adata)
    p=0.5 
    l=spg.search_l(p, adj, start=0.01, end=1000, tol=0.01, max_run=100)
    n_clusters=7
    #Set seed
    r_seed=t_seed=n_seed=100
    #Search for suitable resolution
    res=spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed, t_seed=t_seed, n_seed=n_seed)
    clf=spg.SpaGCN()
    clf.set_l(l)
    #Set seed
    random.seed(r_seed)
    torch.manual_seed(t_seed)
    np.random.seed(n_seed)
    #Run
    clf.train(adata,adj,init_spa=True,init="louvain",res=res, tol=5e-3, lr=0.05, max_epochs=200)
    y_pred, prob=clf.predict()
    adata.obs["pred"]= y_pred
    adata.obs["pred"]=adata.obs["pred"].astype('category')
    #Do cluster refinement(optional)
    #shape="hexagon" for Visium data, "square" for ST data.
    adj_2d=spg.calculate_adj_matrix(x=x_array,y=y_array, histology=False)
    refined_pred=spg.refine(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), dis=adj_2d, shape="hexagon")
    adata.obs["refined_pred"]=refined_pred
    adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')

    obs = adata.obs.copy()
    obs['barcode'] = obs.index
    obs.index = adata.obs[['array_col', 'array_row']].apply(lambda x: 'x'.join(x.astype(str)), axis=1)
    obs.to_csv(f"{out_fd}{tissue_id}/SpaGCN_cluster_results.csv")

  0%|          | 0/12 [00:00<?, ?it/s]

Calculateing adj matrix using xy only...
Run 1: l [0.01, 1000], p [0.0, 3.0902506516689647e+02]
Run 2: l [0.01, 500.005], p [0.0, 84.78437042236328]
Run 3: l [0.01, 250.0075], p [0.0, 21.497591018676758]
Run 4: l [0.01, 125.00874999999999], p [0.0, 4.774121284484863]
Run 5: l [0.01, 62.509375], p [0.0, 0.5137132406234741]
Run 6: l [31.2596875, 62.509375], p [0.00034296512603759766, 0.5137132406234741]
Run 7: l [46.884531249999995, 62.509375], p [0.07667315006256104, 0.5137132406234741]
Run 8: l [54.69695312499999, 62.509375], p [0.24218857288360596, 0.5137132406234741]
Run 9: l [58.6031640625, 62.509375], p [0.36577796936035156, 0.5137132406234741]
Run 10: l [60.556269531249995, 62.509375], p [0.4368997812271118, 0.5137132406234741]
Run 11: l [61.532822265625, 62.509375], p [0.474621057510376, 0.5137132406234741]
recommended l =  62.021098632812496
Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Res =  0.7 Num of clusters 